In [1]:
# Imports needed
import yfinance as yf
import pandas as pd
from pathlib import Path
import datetime
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as fplt

%matplotlib inline

### Useable Functions

#### Get Ticker Data

In [53]:
# Pull in historical data from Yahoo Finance for a specified ticker
import yfinance as yf
import pandas as pd
from pathlib import Path
import datetime
import os

def get_ticker_data(ticker, period, interval):
    '''This function pulls historical data for a specified ticker from Yahoo Finance using yfinance.
    If you do not have yfinance installed, run pip install yfinance
    If the current trading day has not closed it will be removed from the data.
    
    Inputs:
        ticker = ticker symbol for the trade instrument. (str)
        period = data period to download 
            available options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
        interval = data interval.
            available options 1d, 5d, 1wk, 1mo, 3mo     
    '''
    # set folder and file variables
    folder_name = 'Data'
    file_name = ticker
    file = os.path.join(folder_name, file_name + '.csv')
    p = Path(folder_name)
    
    # Get data amd drop the current trading day
    data = yf.Ticker(ticker)
    data_df = data.history(period=period,
                                interval=interval,
                                actions=False,
                                back_adjust=True).reset_index()
    data_df = data_df.iloc[:-1]
    
    # Check if folder and file exist
    if os.path.exists(file):
        
    # Read in historical data CSV, convert dtype of Date column, append DFs, drop dups, and overwrite csv  
        historical_data = pd.read_csv(file)
        historical_data['Date'] = pd.to_datetime(historical_data.Date)
        historical_data = (historical_data.append(data_df)
                  .reset_index()
                  .drop(columns='index')
                 )
        data_df = historical_data.drop_duplicates(keep='last')
        data_df.to_csv(Path(file), index=False) 
        strSuccess = f'Appended data to {file}.'
        
    # Check if directory exist and save df to Data folder as a csv using the ticker as the name
    elif os.path.isdir(folder_name):
        data_df.to_csv(Path(p, ticker + '.csv'), index=False)
        strSuccess = f'Wrote your file to the {p} folder as {file_name}.csv.'
        
    # make the Data folder and save df to csv using the ticker as the name
    else:
        os.makedirs(folder_name)
        data_df.to_csv(Path(p, ticker + '.csv'), index=False)
        strSuccess = f'A folder named, {folder_name}, was created and you file was save in it as {file_name}.csv.'
    return(strSuccess, data_df.tail())

#### Get Twitter Data

In [85]:
# websites used to get twitter accounts to look at
# https://www.offshore-technology.com/features/top-influencers-in-oil-and-gas/
# https://www.benzinga.com/markets/commodities/15/02/5252239/10-oil-experts-to-follow-on-twitter

In [73]:
import tweepy as tw
from tweepy import OAuthHandler
from dotenv import load_dotenv
import json
import csv

def get_twitter_data(screen_name):
    # load data for auth to twitter
    load_dotenv(dotenv_path="C:/Users/brett/Desktop/FTBC/.env")
    api_key = os.getenv('TWITTER_API_KEY')
    api_secret_key = os.getenv('TWITTER_SECRET_KEY')
    bearer_token = os.getenv('TWITTER_BEARER_TOKEN')
    access_token = os.getenv('TWITTER_ACCESS_TOKEN')
    access_s_token = os.getenv("TWITTER_SECRET_TOKEN")
    
    auth = tw.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_s_token)

    api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    # First request to get max count allowed to initialize user_tweets varible to allow iterating to get more tweets
    tweets = []
    total_tweets = 0
    user_tweets = api.user_timeline(screen_name = screen_name,
                                    include_rts=False,
                                    tweet_mode='extended',
                                    count = 200)
    # add user_tweets to tweets list
    tweets.extend(user_tweets)
    total_tweets += len(user_tweets)

    # loop to pull max amount of tweets allowed by twitter (3200)
    while len(user_tweets) > 0:
        # varialbe to be used as max_id for loop
        last_id = user_tweets[-1].id - 1
        
        user_tweets = api.user_timeline(screen_name = screen_name,
                                        max_id = last_id - 1,
                                        include_rts=False,
                                        tweet_mode='extended',
                                        count = 200)
        
        # add additional tweets to original list
        tweets.extend(user_tweets)
        
        print(f'{len(user_tweets)} have been downloaded for {screen_name}')
        total_tweets += len(user_tweets) 
    
    print(f'{total_tweets} downloaded for {screen_name}')
    # extract the data that is needed from the tweet data
    tweet_details = [[tweet.user.screen_name, tweet.created_at, tweet.full_text, tweet.favorite_count, tweet.retweet_count] for tweet in tweets]
    
    # Push extracted data to csv for use later
    with open(f'Data/{screen_name}_tweets.csv', 'w', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(['screen_name', 'date', 'tweet', 'likes', 'retweets'])
        writer.writerows(tweet_details)  

In [30]:
# # code to view the attributes of the tweet data
# import pprint
# status = user_tweets[0]
# # convert to string
# json_str = json.dumps(status._json)
# # deserialise string into python object
# parsed = json.loads(json_str)
# print(json.dumps(parsed, indent=4, sort_keys=True))

#### Plotting 2 Y Axis

In [54]:
def show_plot_twoYs (df, colName0, colName1, colName2):
    #create two axis
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(30,10))
    # make a plot
    ax.plot(df.index, df[colName1], color="red",
#             marker="o",
           )
    # set x-axis label
    ax.set_xlabel('Date',fontsize=14)
    # set y-axis label
    ax.set_ylabel(colName1,color="red",fontsize=14)
    
    # twin object for two different y-axis on the sample plot
    ax2=ax.twinx()
    # make a plot with different y-axis using second axis object
    ax2.plot(df.index, df[colName2],color="blue",
#              marker="o",
            )
    ax2.set_ylabel(colName2,color="blue",fontsize=14)
    plt.show()
    # save the plot as a file
#     fig.savefig(fileName + '.jpg',
#                 format='jpeg',
#                 dpi=100,
#                 bbox_inches='tight')

### Data Prep

#### Load Data from Yahoo

In [2]:
# file paths
cl_path = Path('Data/CL=F.csv')
uup_path = Path('Data/UUP.csv')

# Read files to a dataframe
cl_df = pd.read_csv(cl_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()
uup_df = pd.read_csv(uup_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()

# Drop columns to have just closing price
cl_close = cl_df.drop(columns=['Open', 'High', 'Low', 'Volume'])
uup_close = uup_df.drop(columns=['Open', 'High', 'Low', 'Volume'])

# Change column name to ticker name
cl_close.columns = ['CL']
uup_close.columns = ['UUP']

# # Combine the two dataframes
combined_cl_uup = pd.concat([cl_close, uup_close], axis='columns', join='inner')

#### Load Data from Twitter

In [125]:
# Read in json files
boonepickens_path = Path('Data/boonepickens_tweets.csv')
chrismartenson_path = Path('Data/chrismartenson_tweets.csv')
chrisnelder_path = Path('Data/chrisnelder_tweets.csv')
collineatonhc_path = Path('Data/CollinEatonHC_tweets.csv')
gasbuddyguy_path = Path('Data/GasBuddyGuy_tweets.csv')
jendlouhyhc_path = Path('Data/jendlouhyhc_tweets.csv')
jkempenergy_path = Path('Data/JKempEnergy_tweets.csv')
robinenergy_path = Path('Data/robinenergy_tweets.csv')
staunovo_path = Path('Data/staunovo_tweets.csv')
thearorareport_path = Path('Data/TheAroraReport_tweets.csv')

# Convert to dfs
boonepickens_df = pd.read_csv(boonepickens_path, parse_dates=True, infer_datetime_format=True)
chrismartenson_df = pd.read_csv(chrismartenson_path, parse_dates=True, infer_datetime_format=True)
chrisnelder_df = pd.read_csv(chrisnelder_path, parse_dates=True, infer_datetime_format=True)
collineatonhc_df = pd.read_csv(collineatonhc_path, parse_dates=True, infer_datetime_format=True)
gasbuddyguy_df = pd.read_csv(gasbuddyguy_path, parse_dates=True, infer_datetime_format=True)
jendlouhyhc_df = pd.read_csv(jendlouhyhc_path, parse_dates=True, infer_datetime_format=True)
jkempenergy_df = pd.read_csv(jkempenergy_path, parse_dates=True, infer_datetime_format=True)
robinenergy_df = pd.read_csv(robinenergy_path, parse_dates=True, infer_datetime_format=True)
staunovo_df = pd.read_csv(staunovo_path, parse_dates=True, infer_datetime_format=True)
thearorareport_df = pd.read_csv(thearorareport_path, parse_dates=True, infer_datetime_format=True)

# Combine into 1 dataframe
all_tweets_df = pd.concat([boonepickens_df, chrismartenson_df,
                         chrisnelder_df, collineatonhc_df,
                         gasbuddyguy_df, jendlouhyhc_df,
                         jkempenergy_df, robinenergy_df,
                         staunovo_df, thearorareport_df]).reset_index(drop=True)

# Normalize the Date field
all_tweets_df['date'] = pd.to_datetime(all_tweets_df['date'], errors='coerce')
all_tweets_df['date'] = all_tweets_df['date'].dt.normalize()

# Remove the url from the tweet
tweets_wo_url = []
for tweet in all_tweets_df['tweet']:
    no_url = re.sub(r"http\S+", "", tweet)
    tweets_wo_url.append({
            "tweets_no_url": no_url}
    )
tweets_wo_url_df = pd.DataFrame(tweets_wo_url)

# Append no_url_tweets to all_tweets_df and remove tweets column
all_tweets_df= (pd.merge(all_tweets_df, tweets_wo_url_df, left_index=True, right_index=True, how='inner')
                .drop(columns='tweet')
               )

# Display df
pd.set_option('max_colwidth', 200)
all_tweets_df.head()

,screen_name,date,likes,retweets,tweets_no_url
0,boonepickens,2020-09-11,16,3,"Tune in now for a livestream of the dedication of T. Boone Pickens’ childhood home, the Holdenville House, at his final resting place at Karsten Creek Golf Club in Stillwater, OK. #okstate #Rememb..."
1,boonepickens,2020-09-10,120,24,"Tomorrow is the one-year anniversary of the passing of the legendary T. Boone Pickens. We invite you to a livestream at 12:32 pm CT for the official opening of his childhood home, the Holdenville ..."
2,boonepickens,2020-01-09,37,10,Grateful for the impactful summary of Boone’s final letter @TheRetirementManifesto #spreadtheword – Jay Rosser
3,boonepickens,2020-01-08,96,12,Thank you for this meaningful nod to Boone. He was proud to be associated with such an impactful organization as @NFFNetwork The highlight at the end should bring a chuckle to everyone at #okstate...
4,boonepickens,2019-10-18,53,5,.@FortuneMagazine revisits some of T. Boone Pickens' most memorable quotes throughout the years. #RememberingBoone


### Look at Plots for Ticker Data

In [86]:
cl_df.plot(y='Close')

In [60]:
show_plot_twoYs(combined_cl_uup, combined_cl_uup.index, 'CL', 'UUP')

### Sentiment Analysis

#### Vader Sentiment Analysis

In [146]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

tweets_sentiment = []

for tweet in all_tweets_df['tweets_no_url']:
    try:
        text = tweet
        sentiment = analyzer.polarity_scores(tweet)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]

        tweets_sentiment.append({
            "vader compound": compound,
            "vader positive": pos,
            "vader negative": neg,
            "vader neutral": neu 
        })
    except AttributeError:
        pass

# create Df
vader_df = pd.DataFrame(tweets_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\brett\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [147]:
pd.set_option('max_colwidth', 100)
vader_df.head()

,vader compound,vader negative,vader neutral,vader positive
0,0.4466,0.0,0.914,0.086
1,0.5473,0.0,0.903,0.097
2,0.4588,0.0,0.800,0.200
3,0.9067,0.0,0.699,0.301
4,0.0000,0.0,1.000,0.000


#### Textblob Sentiment Analysis

In [134]:
# Imports needed
from textblob import TextBlob

# Analysis
blob_sentiment = []

for tweet in all_tweets_df['tweets_no_url']:
    blob = TextBlob(tweet)
    blob.sentiment
    polarity = blob.sentiment[0]
    subjectivity = blob.sentiment[1]

    blob_sentiment.append({
        'Textblob Polarity': polarity,
        'Textblob Subjectivity': subjectivity
    })

# create Df
textblob_df = pd.DataFrame(blob_sentiment)
textblob_df.head()

,Textblob Polarity,Textblob Subjectivity
0,0.250000,0.750000
1,0.533333,0.633333
2,0.000000,1.000000
3,0.433333,0.666667
4,0.500000,0.750000


In [150]:
# Merge all_tweets_df with the 2 sentiment dataframes
tweet_sentiment_df = pd.concat([all_tweets_df, vader_df, textblob_df], axis="columns", join='inner')
pd.set_option('max_colwidth', 100)
tweet_sentiment_df.head()

,screen_name,date,likes,retweets,tweets_no_url,vader compound,vader negative,vader neutral,vader positive,Textblob Polarity,Textblob Subjectivity
0,boonepickens,2020-09-11,16,3,"Tune in now for a livestream of the dedication of T. Boone Pickens’ childhood home, the Holdenvi...",0.4466,0.0,0.914,0.086,0.250000,0.750000
1,boonepickens,2020-09-10,120,24,Tomorrow is the one-year anniversary of the passing of the legendary T. Boone Pickens. We invite...,0.5473,0.0,0.903,0.097,0.533333,0.633333
2,boonepickens,2020-01-09,37,10,Grateful for the impactful summary of Boone’s final letter @TheRetirementManifesto #spreadthewor...,0.4588,0.0,0.800,0.200,0.000000,1.000000
3,boonepickens,2020-01-08,96,12,Thank you for this meaningful nod to Boone. He was proud to be associated with such an impactful...,0.9067,0.0,0.699,0.301,0.433333,0.666667
4,boonepickens,2019-10-18,53,5,.@FortuneMagazine revisits some of T. Boone Pickens' most memorable quotes throughout the years....,0.0000,0.0,1.000,0.000,0.500000,0.750000


In [154]:
# Save Df to csv file
tweet_sentiment_df.to_csv('Data/tweet_sentiment_df.csv')

#### IMB Watson Sentiment Analysis

In [73]:
# # Imports needed
# from ibm_watson import ToneAnalyzerV3
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
# from pandas.io.json import json_normalize 
# from dotenv import load_dotenv
# import os
 
# # Get IBM tone keys
# load_dotenv(dotenv_path="C:/Users/brett/Desktop/FTBC/.env")
# tone_api = os.getenv("IBM_TONE_API")
# tone_url = os.getenv("IBM_TONE_URL")

# # Create auth object
# authenticator = IAMAuthenticator(tone_api)

# # Create analyser instances
# tone_analyzer = ToneAnalyzerV3(version="2017-09-21", authenticator=authenticator)

# # Set endpoint
# tone_analyzer.set_service_url(tone_url)

# # Analyze tone of tweets
# for tweet in all_tweets_df['tweets_no_url']:
#     tone_analysis = tone_analyzer.tone(tone_input,
#                                        content_type="text/plan",
#                                        content_language="en",
#                                        accept_language="en").get_result()

# doc_tone_df = json_normalize(data=tone_analysis["document_tone"], record_path=["tones"])
# doc_tone_df.head()

# ibm_tone_df = json_normalize(
#     data=tone_analysis["sentences_tone"],
#     record_path=["tones"],
#     meta=["sentence_id", "text"],
# )
# sentences_tone_df.head()

#### spaCY

In [333]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

tokens = nlp(tweets_str)

# adj = [token.text for token in tokens if token.pos_ == 'ADJ']
# print(adj)

# for token in tokens:
#     print(token.text, token.dep_)

# displacy.render(tokens, style='dep')

### Testing Plots of CandleStick charts

In [59]:
# With Plotly Graph Objects
fig = go.Figure(data=[go.Candlestick(x=cl_df.index,
                open=cl_df['Open'],
                high=cl_df['High'],
                low=cl_df['Low'],
                close=cl_df['Close'])])

fig.show()

In [58]:
# With mplfinance from Matplotlib
fplt.plot(cl_df.iloc[-45:],
         type='candle',
          style='yahoo',
         title='CL Chart',
         ylabel='Price',
          mav=(9,20),
          figratio=(12,5),
          volume=True,
#           savefig='Data/cl_last_45_days.png',
         )

In [17]:
# With Bokeh
from math import pi
from bokeh.plotting import figure
from bokeh.plotting import output_file
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.resources import INLINE

df = cl_df.iloc[-50:]
# df["date"] = pd.to_datetime(df["date"])

# # Needed of using the .rect to make the bars
# mids = (df.Open + df.Close)/2
# spans = df.Close-df.Open

# Calculate 9 and 20 day SMA
short_sma = cl_df['Close'].rolling(window=9).mean()
long_sma = cl_df['Close'].rolling(window=40).mean()

inc = df.Close > df.Open
dec = df.Open > df.Close
w = 12*60*60*1000

# output_file("candlestick.html", title="candlestick.py example")
# output_notebook(resources=INLINE)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=800, plot_height=300, toolbar_location="left", title='Last 50 Days of CL')

# Creates the high and low wick
p.segment(df.index, df.High, df.index, df.Low, color="black")

# Creates the body of the candlestick for the up days
# p.rect(df.index[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.index[inc], w, df.Open[inc], df.Close[inc], fill_color="lawngreen", line_color="black")

# Creates the body of the candlestick foor the down days
# p.vbar(df.index[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")
p.vbar(df.index[dec], w, df.Open[dec], df.Close[dec], fill_color="tomato", line_color="black")

p.xaxis.major_label_orientation = pi/4

# make the grid lines lighter
p.grid.grid_line_alpha=0.3

show(p) 

### Not going to need for project

In [293]:
# Investing.com Data
# file paths
# spgsci_ER_path = Path('Data/spgsci_ER.csv')
# spgsci_TR_path = Path('Data/spgsci_TR.csv')

# Read files to a df
# spgsci_ER_df = pd.read_csv(spgsci_ER_path, thousands=',', index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()
# spgsci_TR_df = pd.read_csv(spgsci_TR_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()

# Change column name Price to Close
# spgsci_ER_df = spgsci_ER_df.rename(columns={'Price': 'Close'})
# spgsci_TR_df = spgsci_TR_df.rename(columns={'Price': 'Close'})

# Drop Volume columns as it is blank
# spgsci_ER_df = spgsci_ER_df.drop(columns='Vol.')
# spgsci_TR_df = spgsci_TR_df.drop(columns='Vol.')

# Drop %Change
# spgsci_ER_df = spgsci_ER_df.drop(columns='Change %')
# spgsci_TR_df = spgsci_TR_df.drop(columns='Change %')

# # Add ticker symbol to df
# spgsci_ER_df['Ticker'] = 'SPGSCLP'
# spgsci_TR_df['Ticker'] = 'SPGSCL'

In [827]:
# Combine two csv files for SP GSCI ER
# # file paths
# spgsci1_path = Path('Data/spgsci_ER_00_19.csv')
# spgsci2_path = Path('Data/spgsci_ER_19_20.csv')

# # Read files to a df
# spgsci1_df = pd.read_csv(spgsci1_path, index_col='Date', parse_dates=True, infer_datetime_format=True, dtype=object).sort_index()        
# spgsci2_df = pd.read_csv(spgsci2_path, index_col='Date', parse_dates=True, infer_datetime_format=True, dtype=object).sort_index()

# # Merge dfs
# spgsci_ER_df = spgsci1_df.combine_first(spgsci2_df)

# # Push df to csv
# spgsci_ER_df.to_csv(r'Data/spgsci_ER.csv')


In [87]:
# # Load Data
# # Read path
# es_path = Path('Data/ES=F.csv')
# uso_path = Path('Data/USO.csv')

# # Read files to a dataframe
# es_df = pd.read_csv(es_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()
# uso_df = pd.read_csv(uso_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()

# # Add ticker symbol to Df
# cl_df["Ticker"] = "CL"
# es_df["Ticker"] = "ES"
# uso_df["Ticker"] = "USO"
# uup_df["Ticker"] = "UUP"

# # Drop columns to have just closing price
# es_close = es_df.drop(columns=['Open', 'High', 'Low', 'Volume'])
# uso_close = uso_df.drop(columns=['Open', 'High', 'Low', 'Volume'])
# spgsci_ER_close = spgsci_ER_df.drop(columns=['Open', 'High', 'Low'])
# spgsci_TR_close = spgsci_TR_df.drop(columns=['Open', 'High', 'Low'])

# # Change column name to ticker name
# es_close.columns = ['ES']
# uso_close.columns = ['USO']
# spgsci_ER_close.columns = ['SPGSCI_ER']
# spgsci_TR_close.columns = ['SPGSCI_TR']

# # combine cl df with other dfs
# combined_cl_es = pd.concat([cl_close, es_close], axis='columns', join='inner')
# combined_cl_uso = pd.concat([cl_close,uso_close], axis='columns', join='inner')
# combined_cl_spgsciER = pd.concat([cl_close,spgsci_ER_close], axis='columns', join='inner')
# combined_cl_spgsciTR = pd.concat([cl_close,spgsci_TR_close], axis='columns', join='inner')

In [92]:
# # Show plot with two Y's access
# # Found these didn't provide any help to the project
# show_plot_twoYs(combined_cl_spgsciER, combined_cl_spgsciER.index, 'CL', 'SPGSCI_ER')
# show_plot_twoYs(combined_cl_spgsciTR, combined_cl_spgsciTR.index, 'CL', 'SPGSCI_TR')
# show_plot_twoYs(combined_cl_uso, combined_cl_uso.index, 'CL', 'USO')